In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import os
import utils

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

INFO 05-03 12:40:26 __init__.py:190] Automatically detected platform cuda.


In [2]:
model_name = "./data/models/Qwen3-8B"
model = LLM(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

INFO 05-03 12:40:42 config.py:542] This model supports multiple tasks: {'classify', 'reward', 'score', 'generate', 'embed'}. Defaulting to 'generate'.
WARNING 05-03 12:40:42 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 05-03 12:40:42 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 05-03 12:40:42 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='./data/models/Qwen3-8B', speculative_config=None, tokenizer='./data/models/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 05-03 12:41:03 model_runner.py:1115] Loading model weights took 15.2585 GB
INFO 05-03 12:41:04 worker.py:267] Memory profiling takes 1.34 seconds
INFO 05-03 12:41:04 worker.py:267] the current vLLM instance can use total_gpu_memory (47.53GiB) x gpu_memory_utilization (0.90) = 42.78GiB
INFO 05-03 12:41:04 worker.py:267] model weights take 15.26GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 26.06GiB.
INFO 05-03 12:41:04 executor_base.py:110] # CUDA blocks: 11859, # CPU blocks: 1820
INFO 05-03 12:41:04 executor_base.py:115] Maximum concurrency for 40960 tokens per request: 4.63x
INFO 05-03 12:41:06 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:16<00:00,  2.09it/s]

INFO 05-03 12:41:23 model_runner.py:1562] Graph capturing finished in 17 secs, took 0.71 GiB
INFO 05-03 12:41:23 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 19.89 seconds


In [3]:
import utils

In [4]:
math_500 = utils.read_from_json("./data/Math-500.json")

In [5]:
math_500[0]

{'problem': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$',
 'solution': 'We have that $r = \\sqrt{0^2 + 3^2} = 3.$  Also, if we draw the line connecting the origin and $(0,3),$ this line makes an angle of $\\frac{\\pi}{2}$ with the positive $x$-axis.\n\n[asy]\nunitsize(0.8 cm);\n\ndraw((-0.5,0)--(3.5,0));\ndraw((0,-0.5)--(0,3.5));\ndraw(arc((0,0),3,0,90),red,Arrow(6));\n\ndot((0,3), red);\nlabel("$(0,3)$", (0,3), W);\ndot((3,0), red);\n[/asy]\n\nTherefore, the polar coordinates are $\\boxed{\\left( 3, \\frac{\\pi}{2} \\right)}.$',
 'answer': '\\left( 3, \\frac{\\pi}{2} \\right)',
 'subject': 'Precalculus',
 'level': 2,
 'unique_id': 'test/precalculus/807.json'}

In [6]:
prompt = """@ Question-answering problem:
```md
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
```

@ Existence problem:
```md
Prove the existence of the point in polar coordinates $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$ for the point $(0,3)$ in rectangular coordinates.
```

===

@ Question-answering problem:
```md
Define
\[p = \sum_{k = 1}^\infty \frac{1}{k^2} \quad \text{and} \quad q = \sum_{k = 1}^\infty \frac{1}{k^3}.\]Find a way to write
\[\sum_{j = 1}^\infty \sum_{k = 1}^\infty \frac{1}{(j + k)^3}\]in terms of $p$ and $q.$
```

@ Existence problem:
```md
Define
\[p = \sum_{k = 1}^\infty \frac{1}{k^2} \quad \text{and} \quad q = \sum_{k = 1}^\infty \frac{1}{k^3}.\] Prove that \[\sum_{j = 1}^\infty \sum_{k = 1}^\infty \frac{1}{(j + k)^3}\] can be represented in terms of $p$ and $q$.
```

===

@ Question-answering problem:
```md
If $f(x) = \frac{3x-2}{x-2}$, what is the value of $f(-2) +f(-1)+f(0)$? Express your answer as a common fraction.
```

@ Existence problem:
```md
Define $f(x) = \frac{3x-2}{x-2}$. Prove that there exists $ y = f(-2) + f(-1) + f(0)$ and $y$ can be expressed as a common fraction.
```

===""" + f"""

@ Question-answering problem:
```md
{math_500[0]['problem']}
```

Based on the above examples, translate the QA problem into existence proble then, put it in a markdown code block as the examples."""

messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True, 
    enable_thinking=True
)

print(text)

<|im_start|>user
@ Question-answering problem:
```md
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates. Enter your answer in the form $(r,	heta),$ where $r > 0$ and $0 \le 	heta < 2 \pi.$
```

@ Existence problem:
```md
Prove the existence of the point in polar coordinates $(r,	heta),$ where $r > 0$ and $0 \le 	heta < 2 \pi.$ for the point $(0,3)$ in rectangular coordinates.
```

===

@ Question-answering problem:
```md
Define
\[p = \sum_{k = 1}^\infty rac{1}{k^2} \quad 	ext{and} \quad q = \sum_{k = 1}^\infty rac{1}{k^3}.\]Find a way to write
\[\sum_{j = 1}^\infty \sum_{k = 1}^\infty rac{1}{(j + k)^3}\]in terms of $p$ and $q.$
```

@ Existence problem:
```md
Define
\[p = \sum_{k = 1}^\infty rac{1}{k^2} \quad 	ext{and} \quad q = \sum_{k = 1}^\infty rac{1}{k^3}.\] Prove that \[\sum_{j = 1}^\infty \sum_{k = 1}^\infty rac{1}{(j + k)^3}\] can be represented in terms of $p$ and $q$.
```

===

@ Question-answering problem:
```md
If $f(x) = rac{3x-2}{x-2}$, what 

In [ ]:
sampling_params = SamplingParams(temperature=0.6, top_p=0.95, max_tokens=8096)
output = model.generate(text, sampling_params=sampling_params)
output_text = output[0].outputs[0].text
print(output_text)

Processed prompts: 100%|██████████| 1/1 [00:19<00:00, 19.21s/it, est. speed input: 31.14 toks/s, output: 35.15 toks/s]


In [9]:
output_text = output[0].outputs[0].text
print(output_text)

<think>
Okay, let's tackle this problem step by step. The user wants me to convert the rectangular coordinates (0, 3) to polar coordinates, ensuring that r is positive and θ is between 0 and 2π. 

First, I remember that converting from rectangular (x, y) to polar (r, θ) involves using the formulas r = √(x² + y²) and θ = arctan(y/x). But I need to be careful with the quadrant where the point lies. 

The given point is (0, 3). Let me visualize this. In the rectangular coordinate system, x is 0 and y is 3, so this is a point on the positive y-axis. 

Calculating r first: r = √(0² + 3²) = √(0 + 9) = √9 = 3. So r is 3, which is positive as required.

Now for θ. Since x is 0 and y is positive, the point is on the positive y-axis. The arctangent of y/x would be arctan(3/0), but division by zero is undefined. However, I know that when x is 0 and y is positive, the angle is π/2 radians (90 degrees). 

But wait, I should confirm that θ is in the correct range. The standard range for θ is 0 ≤ θ <